In [2]:
#1st we do orthagonalisation by Arnoldi method
#Input - A(n*n) matrix , v(n vector) , m (a positive integer)
#(m+1) orthogonal vectors, Hessenberg matrix
#orthogonal vectors spanning the krylov space
#breaks when degree of minimal polynomial is achieved

In [3]:
import numpy as np
from numpy import linalg 

In [19]:
def arnoldi(A,m):
    n = A.shape[0]
    V = np.zeros([n,m+1])
    H = np.zeros([m+1,m])
    v = np.random.rand(n)
    V[:,0] = v/(np.linalg.norm(v))
    for k in range(m):
        vcap = A.dot(V[:,k])
        for j in range(k+1):
            H[j,k] = (V[:,j].T).dot(vcap)
            vcap = vcap - H[j,k]*V[:,j]
        H[k+1,k] = np.linalg.norm(vcap)
        if (H[k+1,k]<1e-10):
            print("breakdown")
            break
        else:
            V[:,k+1] = vcap/H[k+1,k]
    return V, H

In [20]:
A = np.random.rand(5,5)

In [26]:
D = np.dot(A,A.T)
D1= D-np.eye(5)

In [27]:
V1,H1 = arnoldi(D,3)
V2,H2 = arnoldi(D1,3)

In [28]:
V1

array([[ 0.51952501, -0.0357854 , -0.22056095, -0.59879982],
       [ 0.16284528,  0.50555957,  0.28675856,  0.55110028],
       [ 0.43798641,  0.26087779, -0.72751146,  0.33646734],
       [ 0.65779544, -0.53655455,  0.41083211,  0.27683146],
       [ 0.2811552 ,  0.62223864,  0.41360181, -0.38455341]])

In [29]:
V2

array([[ 0.38940917,  0.21716949,  0.15678382,  0.41556766],
       [ 0.16801911,  0.60282109, -0.4191128 , -0.63506462],
       [ 0.27940879,  0.61284056,  0.05364107,  0.46597188],
       [ 0.38468402, -0.00166212,  0.79579508, -0.45462451],
       [ 0.77076521, -0.46245847, -0.40447017, -0.01353553]])

In [4]:
D = np.zeros((10, 10), int)
np.fill_diagonal(D, 1)
D[0,9] = 1
D[9,0] = 1
v = np.random.randint(10,size=10)
n = 10
A0 = np.random.rand(n,n) 
A = A0 @ A0.T
v = np.random.rand(n)
#The D proposed only three orthogonal vector

In [5]:
V, H = arnoldi(A0,10,v)

In [6]:
H.shape

(11, 10)

In [8]:
for i in range(7):
    for j in range(7):
        #print(V[:,i].dot(V[:,j]))
    print("---------------")

IndentationError: expected an indented block (<ipython-input-8-5e0283ec6bc3>, line 4)

In [9]:
#Full orthogonal method
#Now we will solve the equation Ax = b by Arnoldi method using Galerkin Method
#r0 = b-Ax0
#xm = x0 + zm
#zm= Vm.Ym
#let v1 = r0/norm(r0)
#find H(m+1) V(m+1)
#solve Hm.Ym = norm(r0)e1
#residual rm = -h(m+1,m)em.T.Ym.v(m+1)
#norm(rm) = h(m+1,m)|em.T.Ym|

In [10]:
#Explicitly Restarted Arnoldi method (FOM) for Ax = b
#input : A(n,n), m, e, x0
#output : xm such that rm is orthogobnal to krylov space
#As the matrix is Hessenberg so we will decompose it into LU nad then use forward and backward substitution

In [11]:
#given a hessenberg matrix we need tofind the solution
#convert it into a upper triangular matrix
def hessol(A,b):
    m = A.shape[0]
    x = np.zeros([m,])
    #print(A)
    for i in range(1,m):
        #print(A[i-1,:].shape)
        A[i,:] = A[i,:] - A[i-1,:]*(A[i,i-1]/A[i-1,i-1])
        b[i] = b[i] - b[i-1]*(A[i,i-1]/A[i-1,i-1])
   # print(A)
    for j in range (m-1,-1,-1):
        temp = b[j]
        for k in range(m-1,j,-1):
            temp = temp - A[j,k]*x[k] 
        x[j] = temp/A[j,j]
    return x

In [12]:
A = np.random.rand(4,4)
A[2,0] = 0 
A[3,0] = 0 
A[3,1] = 0 
b = np.random.rand(4)
x = hessol(A,b)
print(x)
print(A@x)
print(b)

[-18.48111792  20.96676837  34.00388203 -24.53529755]
[ 0.83733894  0.09644537  0.76521767  0.89729001]
[ 0.83733894  0.09644537  0.76521767  0.89729001]


In [35]:
def FOM(A,m,e,x0,b,p,S=0,c=0):
    if(S==0):
        r0 = b - A.dot(x0)
    else:
        r0 = c
    temp = np.linalg.norm(r0)
    #print(temp)
    v = r0/temp
    V, H = arnoldi(A,m,v)
    #print(H[m,m-1])
    E = np.zeros(m)
    E[0] = 1
    #print(H[:m,:])
    #print(temp*E)
    y = hessol(H[:m,:],temp*E)
    zm = V[:,:m]@y
    x0 = x0 - zm
    E = np.zeros(m)
    E[m-1] = 1
    temp2 = np.dot(E,y)
    rm = -1*H[m,m-1]*temp2*V[:,m]
    residual = np.linalg.norm(rm)
    #print(np.linalg.norm((b-A@x0)))
    print((x0))
    print(p)
    #print(residual)
    #print(temp2)
    #print(H[m,m-1])
    if(p < 0):
        return x0
    else:
        x0 = FOM(A,m,e,x0,b,p-1,1,rm)

In [36]:
A = np.diag(range(1,101))
A[0,99] = 1
A[99,0] = 1
B = np.random.rand(5,5)
b = np.random.rand(5)
v = np.random.rand(5)
print(np.linalg.norm((b-B@v)))
print("-----")
x = FOM(B,5,0.00000001,v,b,10,0,0)
#print(B)
#print(b)

1.99182487909
-----
[ 0.82046432  1.20394226  0.88750551  0.23159583  1.60100008]
10
[ nan  nan  nan  nan  nan]
9
[ nan  nan  nan  nan  nan]
8
[ nan  nan  nan  nan  nan]
7
[ nan  nan  nan  nan  nan]
6
[ nan  nan  nan  nan  nan]
5
[ nan  nan  nan  nan  nan]
4
[ nan  nan  nan  nan  nan]
3
[ nan  nan  nan  nan  nan]
2
[ nan  nan  nan  nan  nan]
1
[ nan  nan  nan  nan  nan]
0
[ nan  nan  nan  nan  nan]
-1


/home/stark/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
